In [3]:
# from huggingface_hub import notebook_login
# notebook_login()


In [1]:
# from huggingface_hub import HfApi
# available_pipelines = [p.modelId for p in HfApi().list_models(filter="pyannote-audio-pipeline")]
# list(filter(lambda p: p.startswith("pyannote/"), available_pipelines))

import whisper, json, datetime
start = datetime.datetime.now()
from pyannote.audio import Pipeline
  
# pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token=True)
# pipeline = Pipeline.from_pretrained('pyannote/speaker-diarization@2.1', use_auth_token=True)
pipeline = Pipeline.from_pretrained("pyannote/speaker-diarization", use_auth_token="")

# Check si CUDA ok
import torch
print(torch.cuda.is_available())
print(f'{datetime.datetime.now() - start}')

ModuleNotFoundError: No module named 'pyannote'

In [2]:
# pathfile="./AlainWOrk/140322/audio2.wav"
pathfile="./Anger/anger_26-09-2016.wav"

# Creation des dossiers pour l'enregistrement des resultats 
import pathlib, os, json
path = pathlib.Path(pathfile)
dir_result = f"{path.parent.absolute()}/{path.name}_result"

pathlib.Path(f"{dir_result}").mkdir(parents=True, exist_ok=True)

os.listdir(path.parent.absolute())

['anger_26-09-2016.wav_result']

In [4]:
start = datetime.datetime.now()
diarization_result = pipeline(pathfile)

with open(f"{dir_result}/{path.name}.diarization.json", 'w') as d:
    json.dump(diarization_result.for_json(), d, ensure_ascii=False, indent=4)
d.close()

with open(f"{dir_result}/{path.name}.diarization.txt", 'w') as t:
    t.write(f"{diarization_result}")
t.close()

print(f'diarization_result ok in {datetime.datetime.now() - start}')

diarization_result ok in 0:13:01.557050


In [ ]:
start = datetime.datetime.now()
model = whisper.load_model("large-v2")
print(f"Model chargé en {datetime.datetime.now() - start} Fait chauffer le gpu Marcel ! ")

start = datetime.datetime.now()
whisper_res = model.transcribe(pathfile, verbose=True, language="French")
print(f"whisper_res OK in {datetime.datetime.now() - start}")

with open(f"{dir_result}/{path.name}.whisper.json", 'w') as f:
    json.dump(whisper_res, f, ensure_ascii=False, indent=4)
f.close()

In [3]:
from pyannote.core import Segment, Annotation, Timeline


def get_text_with_timestamp(transcribe_res):
    timestamp_texts = []
    for item in transcribe_res['segments']:
        start = item['start']
        end = item['end']
        text = item['text']
        timestamp_texts.append((Segment(start, end), text))
    return timestamp_texts


def add_speaker_info_to_text(timestamp_texts, ann):
    spk_text = []
    for seg, text in timestamp_texts:
        spk = ann.crop(seg).argmax()
        spk_text.append((seg, spk, text))
    return spk_text


def merge_cache(text_cache):
    sentence = ''.join([item[-1] for item in text_cache])
    spk = text_cache[0][1]
    start = text_cache[0][0].start
    end = text_cache[-1][0].end
    return Segment(start, end), spk, sentence


PUNC_SENT_END = []
# PUNC_SENT_END = ['.', '?', '!']


def merge_sentence(spk_text):
    merged_spk_text = []
    pre_spk = None
    text_cache = []
    for seg, spk, text in spk_text:
        if spk != pre_spk and pre_spk is not None and len(text_cache) > 0:
            merged_spk_text.append(merge_cache(text_cache))
            text_cache = [(seg, spk, text)]
            pre_spk = spk

        elif text[-1] in PUNC_SENT_END:
            text_cache.append((seg, spk, text))
            merged_spk_text.append(merge_cache(text_cache))
            text_cache = []
            pre_spk = spk
        else:
            text_cache.append((seg, spk, text))
            pre_spk = spk
    if len(text_cache) > 0:
        merged_spk_text.append(merge_cache(text_cache))
    return merged_spk_text


def diarize_text(transcribe_res, diarization_result):
    timestamp_texts = get_text_with_timestamp(transcribe_res)
    spk_text = add_speaker_info_to_text(timestamp_texts, diarization_result)
    res_processed = merge_sentence(spk_text)
    return res_processed


def write_to_txt(spk_sent, file):
    with open(file, 'w') as fp:
        for seg, spk, sentence in spk_sent:
            line = f'{seg.start:.2f} {seg.end:.2f} {spk} {sentence}\n'
            fp.write(line)
        
        
def add_space_before_punctuation(text):
    text = re.sub(r'(\w)([?!])', r'\1 \2', text)
    return text

def remove_space_and_tiret(text):
    text = re.sub(r'^\ ', '',text)
    text = re.sub(r'^\— ', '',text)
    return text

def replace_words(text):

    word_dict = {
        "que l'on doit": "que nous devons",
        "on propose": "nous proposons",
        "on restait": "nous restions",
        "on restera": "nous resterons",
        "on reste": "nous restons",
        "on reprend": "nous reprenons",
        "qu'on s'était": "que nous nous étions",
        "qu'on s'est": "que nous nous sommes",
        "on se rend": "nous nous rendons",
        "on vous suit": "nous vous suivons",
        "on pourrait": "nous pourrions",
        "on peut": "nous pouvons",
        "on ne saura": "nous ne saurons",
        "on peut s'en": "nous pouvons nous en ",
        "il va falloir": "il faudra",
        "on n'avait": "nous n'avions ",
        "qu'on ferait": "que nous ferions",
        "c'est des": "ce sont des",
        "c'était pas": "ce n'était pas",
        "j'ai pas": "je n'ai pas",
        "il a pas ": "il n'a pas",
        "c'est des": "ce sont des",
        "qu'on n'est plus": "que nous ne sommes plus",
        "qu'on n'est": "que nous sommes",
        "qu'on est": "que nous sommes",
        "qu'on a": "que nous avons",
        "qu'on n'a plus": "que nous n'avons plus",
        "on en avait": "nous en avions",
        "on en a": "nous en avons",
        "on tenait": "nous tenions",
        "il faut pas": "il ne faut pas",
        "il doit pas": "il ne doit pas",
        "il travaille pas": "il ne travaille pas",
        "on n'est pas": "nous ne sommes pas",
        "qu'on puisse": "que nous puissions",
        "est-ce que vous avez": "avez-vous",
        "est-ce que vous êtes ": "etes-vous",
        "est-ce qu'il serait": "serait-il",
        "on ne s'est pas": "nous ne nous sommes pas",
        "qu'on n'était pas": "que nous n'étions pas",
        "qu'on était pas": "que nous n'étions pas",
        "qu'on ne mettait pas": "que nous ne mettions pas",
        "qu'on mettait pas": "que nous ne mettions pas",
        "on peut se": "nous pouvons nous",
        "qu'on en a": "que nous en avons",
        "est-ce qu'on peut": "pouvons-nous",
        "on a": "nous avons",
        "on est": "nous sommes",
        "on vient": "nous venons",
        "on va": "nous allons",
        "on obtient": "nous obtenons",
        "on fait": "nous faisons",
        "on coordonne": "nous coordonnons",
        "on se frotte": "nous nous frottons",
        "on se donne": "nous nous donnons",
        "on mange": "nous mangeons",
        "on donne": "nous donnons",
        "on apprend": "nous apprenons",
        "on s'est dit": "nous nous sommes dit",
        "on sait": "nous savons",
        "on prend": "nous prenons",
        "ça": "cela",
        "est-ce que vous avez": "avez-vous",
        "est-ce qu'il y a": "y a-t-il ",
        "puisqu'on voit": "puisque nous voyons",
        "qu'on voit": "que nous voyons",
        "comment on peut": "comment pouvons-nous",
        "on participe": "nous participons",
        "-là": "",
        "qu'nous": "que nous",
        "tu ne m'as pas": "vous ne m'avez pas",
        "tu as": "vous avez",
        "tu fais": "vous faites",
        "tu es": "vous êtes",
        "tu dis": "vous dites",
        "tu parles": "vous parlez",
        "tu rentres": "vous rentrez",
        "tu avais": "vous aviez",
        "tu n'avais pas": "vous n'aviez pas",
    }
    

    for old_word, new_word in word_dict.items():
        pattern = r'\b(' + old_word + r')\b'                   
        pattern_case_insensitive = re.compile(pattern, re.IGNORECASE)
        text = remove_space_and_tiret(text)
        text = add_space_before_punctuation(text)
        text = re.sub(pattern_case_insensitive, lambda m: new_word.capitalize() if m.group(1)[0].isupper() else new_word.lower(), text)
    return text                                                             
                        

In [ ]:
import re

# pathfile="./jmarc/jmarc.wav"
# path = pathlib.Path(pathfile)
# dir_result = f"{path.parent.absolute()}/{path.name}_result"

# with open(f"{dir_result}/{path.name}.whisper.json", 'r') as f:
#     whisper_res_from_json = json.load(f)

# with open(f"{dir_result}/{path.name}.diarization.json", 'r') as d:
#     diarization_result_fron_json = json.load(d)
    
# print(type(diarization_result))
# print(type(diarization_result_fron_json))
# print(type(whisper_res))
# print(type(whisper_res_from_json))
    
final_result = diarize_text(whisper_res, diarization_result)
with open(f"{dir_result}/{path.name}.final.txt", 'w') as r:
    ex_speaker = None
    for segment, speaker, text in final_result:
        if speaker != ex_speaker:
            r.write('\n\n')
            r.write(f'{speaker}')
            r.write('\n\n')
            ex_speaker = speaker
        r.write(replace_words(text))
r.close()

# final = open(f"{dir_result}/{path.name}.final.txt", 'rt')
# print(final.read())


In [37]:
speakers_list = []

list_txt = []
ex_speaker = None
text = {'whisper_text' : '', 'replace_text' : ''}
for segment, speaker, original_text in final_result:
    if speaker != ex_speaker:
        list_txt.append(text)
        text = {
            'speaker': f'{speaker}', 
            'segment' : f'{segment}', 
            'whisper_text' : '', 
            'replace_text': '',
        }
        ex_speaker = speaker
        speakers_list.append(speaker)
    text['whisper_text'] += original_text
    text['replace_text'] += replace_words(original_text)


    
result_dict = {f'{pathfile}' : {
                   "speakers": [(speaker, '') for speaker in list(set(speakers_list))],
                   "results" : list_txt
                    }
              }

with open(f"{dir_result}/{path.name}.final_dict.json", 'w') as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=4)
f.close()

In [12]:
import os
import openai

openai.api_key = ""

def gpt3edit(instruction=None, prompt=None):
    response = openai.Edit.create(
      model="text-davinci-edit-001",
      input=f"{prompt}",
      instruction=f"{instruction}",
      temperature=0,
      top_p=1
    )
    return response['choices'][0]['message']['content']


def gpt35turbo(system=None, prompt=None):
    response = openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
            {"role": "system", "content": f"{system}"},
            {"role": "user", "content": f"{prompt}"},
        ]
    )
    return response['choices'][0]['message']['content']

edit = "corrige les fautes d'orthographe, de grammaire et de syntaxe."

Correction = "Tu dois corriger les fautes d'orthographe et de syntaxe comme le fait une stenographe "\
"tout en gardant le texte similaire à la réalité. "\
"Aucun résumé, aucun avis, tu dois rester EXTREMEMENT fidèle au texte d'origine. " \
"Le sens ne doit jamais être altéré. "

Resume = "Tu es chargé du compte rendu de réunions."\
         "Résume chaque intervention"\
         " par une liste des arguments, idées ou pensée de cette façon : "\
        " - idée 1"\
        " - idée 2"\
        " - idée etc ...."\

classification = "Classification de texte : Veuillez classer le texte fourni dans une catégorie spécifique en fonction de son contenu. Les catégories peuvent inclure des sujets tels que les actualités, la politique, la technologie, la santé, l'éducation, etc."

entite = "Veuillez analyser le texte fourni et identifier toutes les entités nommées telles que les personnes, les organisations, les lieux, les dates, etc."

sentiment = "Veuillez analyser le texte fourni et déterminer l'attitude générale exprimée dans le texte. Le texte peut être positif, négatif ou neutre."

In [13]:
from transformers import GPT2TokenizerFast
tokenizer = GPT2TokenizerFast.from_pretrained("gpt2")
# tokenizer("Hello world")['input_ids']

with open(f"/home/slaan/Gits/SummerWhisper/anger_26-09-2016.wav_result/anger_26-09-2016.wav.final_dict.json", 'r', encoding='utf8') as f:
    result_dict_loaded = json.load(f)
f.close()
list_txt = result_dict_loaded[f"{pathfile}"]['results']
print(len(list_txt))


for txt in list_txt[26:29]:
    txt['num'] = list_txt.index(txt)
    txt['token'] = len(tokenizer(txt['whisper_text'])['input_ids'])
    if txt['token'] > 60 :
        print(f'{list_txt.index(txt)} / {len(list_txt)}')
        # txt['STENO'] = gpt35turbo(system=Correction, prompt=f"{txt['whisper_text']}")
        # txt['RESUME'] = gpt35turbo(system=Resume, prompt=f"{txt['whisper_text']}")
        # txt['CLASSIFICATION']= gpt35turbo(system=classification, prompt=f"{txt['STENO']}")
        # txt['ENTITE']= gpt35turbo(system=entite, prompt=f"{txt['STENO']}")
        # txt['SENTIMENT']= gpt35turbo(system=sentiment, prompt=f"{txt['STENO']}")
        txt['EDIT']= gpt3edit(instruction=edit, prompt=f"{txt['whisper_text']}")
#         print("")
#         print("ORIGINAL")
#         print(txt.get('speaker'))
#         print(txt.get('whisper_text'))
#         print("OCCURENCE")
#         print(txt.get('replace_text'))
#         print("TURBO STENO")
#         print(txt['STENO'] )
#         print("TURBO RESUME")
#         print(txt['RESUME'])
#         print("")


Token indices sequence length is longer than the specified maximum sequence length for this model (1917 > 1024). Running this sequence through the model will result in indexing errors


247
27 / 247


InvalidRequestError: The model: `text-davinci-edit-001` does not exist

In [ ]:
# TODO: https://github.com/promptslab/Promptify
    

In [98]:
for txt in list_txt:
    txt['num'] = list_txt.index(txt)
    
result_dict = {f'{pathfile}' : {
                   "speakers": [(speaker, '') for speaker in list(set(speakers_list))],
                   "results" : list_txt
                    }
              }

with open(f"{dir_result}/{path.name}.final_dict.json", 'w', encoding=("utf8")) as f:
    json.dump(result_dict, f, ensure_ascii=False, indent=4)
